In [ ]:
!pip install faiss-cpu
!pip install open_clip_torch

In [ ]:
import os
import glob
import faiss
import numpy as np
from tqdm import tqdm
import open_clip
import torch
import json
import cv2 
import matplotlib.pyplot as plt
from PIL import Image 

In [ ]:
with open('/kaggle/input/link-faiss/output.json') as json_file:
    data_json = json.load(json_file)

In [ ]:
a = np.load('/kaggle/input/clip-feature-aic/CLIPv2_features/L27/V001.npy')
a.shape

In [ ]:
feature_shape = 1024
features_dir = '/kaggle/input/clip-feature-aic/CLIPv2_features'
print(features_dir)
index = faiss.IndexFlatIP(feature_shape)
d = 0
for data_part in tqdm(sorted(os.listdir(features_dir))):
    for feature_path in tqdm(sorted(glob.glob(os.path.join(features_dir, data_part) +'/*.npy'))):
        print(feature_path)
        feats = np.load(feature_path)
        for feat in feats:
            d= d+1
            feat = feat.astype(np.float32).reshape(1,-1)
            index.add(feat)

faiss.write_index(index, f"faiss_clipv2_cosine.bin")
print(d)

In [ ]:
index = faiss.read_index('/kaggle/working/faiss_clipv2_cosine.bin')

In [ ]:
clipv2_tokenizer = open_clip.get_tokenizer('ViT-H-14-378-quickgelu')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
clipv2_model, _, _ = open_clip.create_model_and_transforms('ViT-H-14-378-quickgelu', device=device, pretrained='dfn5b')

In [ ]:
text = clipv2_tokenizer(["airport inside"]).to(device)  
text_features = clipv2_model.encode_text(text)
text_features /= text_features.norm(dim=-1, keepdim=True)
text_features = text_features.cpu().detach().numpy().astype(np.float32)

In [ ]:
k = 10
scores, idx_image = index.search(text_features, k=k)
idx_image = idx_image.flatten()
print(scores)
# for i in list(idx_image):
#     print(data_json[f'{i}'])
#     img = cv2.imread(data_json[f'{i}'])
#     img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
#     plt.imshow(img)
#     print(i)
num_images = len(list(idx_image))
rows = num_images
cols = 1

fig, axes = plt.subplots(rows, cols, figsize=(48, 24))
for i, ax in enumerate(axes):
#     print(data_json[f'{idx_image[i]}'])
    img = cv2.imread(data_json[f'{idx_image[i]}'])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    ax.imshow(img)
    ax.axis('off')  # Tắt trục x và y

plt.show()

In [ ]:
data_json['0']

In [ ]:
if index is None:
  scores, idx_image = index_choosed.search(text_features, k=k)
else:
  id_selector = faiss.IDSelectorArray(index)
  scores, idx_image = index_choosed.search(text_features, k=k, params=faiss.SearchParametersIVF(sel=id_selector))
idx_image = idx_image.flatten()
idx_image
###### GET INFOS KEYFRAMES_ID ######
# infos_query = list(map(self.id2img_fps.get, list(idx_image)))
# image_paths = [info['image_path'] for info in infos_query]


In [ ]:
index